In [1]:
import pandas as pd
import numpy as np
import random
from particle_swarm import particle_swarm_optimization, hybrid_particle_swarm_optimization

In [2]:
random.seed(10)
np.random.seed(10)

In [3]:
citiesnp = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1) # load as numpy array

In [6]:
res = particle_swarm_optimization(path_length=len(citiesnp)-1,
                                  cities=citiesnp,
                                  inertia_weight=0.9)
res[-1]

Iteration: 10, minimum distance so far: 445745002.0475904
Iteration: 20, minimum distance so far: 445654486.50766546
Iteration: 30, minimum distance so far: 445654486.50766546
Iteration: 40, minimum distance so far: 445654486.50766546
Iteration: 50, minimum distance so far: 445628826.0015091
Iteration: 60, minimum distance so far: 445595704.7796311
Iteration: 70, minimum distance so far: 445584457.454726
Iteration: 80, minimum distance so far: 445583234.63340247
Iteration: 90, minimum distance so far: 445573613.10792744
Iteration: 100, minimum distance so far: 445546515.9129143
Iteration: 110, minimum distance so far: 445544151.11928874
Iteration: 120, minimum distance so far: 445544151.11928874
Iteration: 130, minimum distance so far: 445544151.11928874
Iteration: 140, minimum distance so far: 445540158.420273
Iteration: 150, minimum distance so far: 445535200.66557944
Iteration: 160, minimum distance so far: 445531737.764438
Iteration: 170, minimum distance so far: 445531683.3492059


[349, 445507331.1803256]

## Hybrid particle swarm

In [4]:
res = hybrid_particle_swarm_optimization(path_length=len(citiesnp)-1,
                                  cities=citiesnp,
                                  inertia_weight=0.9)
res[-1]

Iteration: 50, minimum distance so far: 445224067.5520419
Iteration: 100, minimum distance so far: 445181865.64914566
Iteration: 150, minimum distance so far: 445158085.5280965
Iteration: 200, minimum distance so far: 445145265.4297618
Iteration: 250, minimum distance so far: 445120282.6343104
Iteration: 300, minimum distance so far: 445095115.31729907
Iteration: 350, minimum distance so far: 445057981.5254689
Iteration: 400, minimum distance so far: 445048335.1714868
Iteration: 450, minimum distance so far: 445035298.28880507
Iteration: 500, minimum distance so far: 445016969.54869777
Iteration: 550, minimum distance so far: 445010102.9228304
Iteration: 600, minimum distance so far: 444991612.22048765
Iteration: 650, minimum distance so far: 444986561.2615773


[652, 444986561.2615773]